In [15]:
#Imagina que o projeto de analise textual que por exemplo categoriza a artigos que são submetidos a um site ou noticias que sejam digitalizadas e automaticamente categorizadas
#então esse processo vai ser automatizado para que uma pessoa não tenha que fazer isso
import pandas as pd
import numpy as np
from sklearn import metrics


import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('C:/csv/news/bbc-text.csv')
df.head()
#aqui já tem as repostas da categoria mas o foco do projeto é não precisar usar dele
#também é necessario transformar o texto em informações numericas para a anlise

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [3]:
#checando as categorias
df.category.value_counts()

category
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64

In [4]:
#listando quantas noticias há de cada um
total_categorias = df.category.value_counts().values
total_categorias

array([511, 510, 417, 401, 386], dtype=int64)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from time import time

#vetorizando as palavras
# max_df =  palavras ou termos que aparecem em mais de 50% dos documentos
#min df =  que aparecem em ao menos 5 documentos

vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=5,
    stop_words="english", #removendo stopwords por não ter valor de informações
)    

In [8]:
#aplicando o vetorizador no conjunto de dados
t0 = time()
X_tfidf = vectorizer.fit_transform(df.text)
print(f"vetorização completa em {time() - t0:.3f} s")
print(f"n_observacoes: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")
#2225 observações e + de 9000 features

vetorização completa em 0.378 s
n_observacoes: 2225, n_features: 9136


In [9]:
#quantas variaveis de fatos foram encontradas?
#Apenas 14% tem valor diferente de 0
print(f"{X_tfidf.nnz / np.prod(X_tfidf.shape):.3f}")

0.014


In [10]:
#diminuindo os valores de 0
from sklearn.cluster import KMeans

for seed in range(5):
    kmeans = KMeans(
        n_clusters=5,
        max_iter=100,
        n_init=1,
        random_state=seed,
    ).fit(X_tfidf)
    cluster_ids, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
    print(f"Número de elementos em cada cluster: {cluster_sizes}")
print()
print(
    "Número de documentos em cada cluster real: "
    f"{total_categorias}"
)

Número de elementos em cada cluster: [298 317 508 821 281]
Número de elementos em cada cluster: [510 779 489 142 305]
Número de elementos em cada cluster: [264 912 490 386 173]
Número de elementos em cada cluster: [758 525 404 169 369]
Número de elementos em cada cluster: [780 540 390 171 344]

Número de documentos em cada cluster real: [511 510 417 401 386]


In [11]:
#redução de dimensionalidade LSA
from sklearn.decomposition import TruncatedSVD #para diminuir o tempo e aumentar a qualidade de analise
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer


lsa = make_pipeline(TruncatedSVD(n_components=100), Normalizer(copy=False)) # Criando um pipeline com TruncatedSVD e Normalizer
t0 = time() # Gravando o tempo antes de iniciar a transformação
X_lsa = lsa.fit_transform(X_tfidf) # Aplicando o pipeline para transformar o conjunto de dados X_tfidf
explained_variance = lsa[0].explained_variance_ratio_.sum() # Calculando a variância explicada pelos componentes principais retidos

print(f"LSA concluido em {time() - t0:.3f} s") # Imprimindo o tempo de execução do LSA
print(f"Variação explicado do SVD: {explained_variance * 100:.1f}%") # Imprimindo a variância explicada pelo SVD truncado

LSA concluido em 0.316 s
Variação explicado do SVD: 26.3%


In [16]:
#rodando algoritmo
#encontrando silhueta
#rand_index = metrica 
#é observado que com k = 5 o index é mais alto o que faz sentido por haver só 5 grupos
for i in range(2,10):
    kmeans_model = KMeans(n_clusters=i, random_state=42).fit(X_lsa)
    labels = kmeans_model.labels_
    print(f"k={i} - silhueta={metrics.silhouette_score(X_lsa, labels, metric='euclidean'):.3f}")
    print(f"k={i} - rand_index={metrics.adjusted_rand_score(df.category.values, labels):.3f}")
    print(20*' * ')
 


k=2 - silhueta=0.041
k=2 - rand_index=0.331
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
k=3 - silhueta=0.050
k=3 - rand_index=0.570
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
k=4 - silhueta=0.059
k=4 - rand_index=0.644
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
k=5 - silhueta=0.070
k=5 - rand_index=0.909
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
k=6 - silhueta=0.072
k=6 - rand_index=0.867
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
k=7 - silhueta=0.075
k=7 - rand_index=0.764
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
k=8 - silhueta=0.075
k=8 - rand_index=0.699
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
k=9 - silhueta=0.079
k=9 - rand_index=0.584
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 


In [17]:
#checando quais palavras impactam mais os grupos
kmeans = KMeans(
    n_clusters=5,
    max_iter=100,
    n_init=1,
    random_state=seed,
).fit(X_lsa)

labels = kmeans.labels_

In [18]:
# gerando clusters de palavras mais frequentes
original_space_centroids = lsa[0].inverse_transform(kmeans.cluster_centers_)
order_centroids = original_space_centroids.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

for i in range(5):
    print(f"Cluster {i}: ", end="")
    for ind in order_centroids[i, :10]:
        print(f"{terms[ind]} ", end="")
    print()

Cluster 0: game england cup club win play players match chelsea injury 
Cluster 1: company growth market shares bank firm economy sales mr oil 
Cluster 2: mr labour election government blair party minister people brown howard 
Cluster 3: film people tv mobile new technology best software digital users 
Cluster 4: music band album best world record olympic singer rock race 


In [19]:
#checando os grupos que estão disponiveis

#por exemplo o cluster 4 está ligado a esporte
#cluster 1 a business
#Cluster 3 Business
#cluster 2 a politica
df.category.unique()

array(['tech', 'business', 'sport', 'entertainment', 'politics'],
      dtype=object)